# Análise de chances de sobrevivencia

> Este notebook aborda a modelagem da chance de sobrevivencia de grupos de pessoas

## Informações do Aluno

* **Nome:** Igor Lima Rocha
* **Matrícula:** 201910282
* **E-mail:** ilrocha.cic@uesc.br

## 1. Introdução

Neste documento, apresento os passos e as técnicas utilizadas para encontrar o melhor modelo de previsão de sobrevivência no desastre do Titanic. O objetivo principal deste trabalho é aplicar técnicas de pré-processamento de dados, regularização e métodos de classificação para criar um modelo preciso.

O conjunto de dados utilizado nesta análise é proveniente da competição ["Titanic: Machine Learning from Disaster"](https://www.kaggle.com/competitions/titanic/overview) no Kaggle, que oferece um conjunto desafiador de dados relacionados à sobrevivência dos passageiros. Este relatório descreverá em detalhes cada etapa do processo, desde a limpeza dos dados até a regularização, modelagem e avaliação final do modelo.

## 2. Carregamento dos Dados

Agora, vamos carregar o conjunto de dados em um DataFrame para começar nossa análise.

In [84]:
import pandas as pd

# Carregar o dataset
titanic_data = pd.read_csv("train.csv")

# Salvando o dataset original como backup, para avaliações futuras
df_original = titanic_data.copy()

titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Com o DataFrame carregado, podemos prosseguir para as próximas etapas.

## 3. Checagem dos Tipos de Dados

É crucial conhecer os tipos de dados com os quais estamos trabalhando. Vamos dar uma olhada.

In [85]:
titanic_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Com isso, podemos ver que o conjunto de dados contém diversos tipos de dados.

Com isso em mente, vamos começar a tratá-los, removendo valores ausentes e codificando tipos categóricos.

## 4. Limpeza dos Dados

### 4.1 Identificando Valores Faltantes

A primeira etapa da limpeza de dados envolve o tratamento de valores ausentes.

In [86]:
titanic_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Como podemos perceber, existem valores faltantes nas colunas `Age`, `Cabin` e `Embarked`.

#### 4.1.1 Valores Faltantes na Coluna 'Age'

Uma abordagem comum é substituir os valores faltantes pela média dos valores existentes. No entanto, também é possível utilizar a mediana ou até mesmo um valor constante, dependendo do contexto.

In [87]:
# Usando a média
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)

#### 4.1.2 Outras colunas

Dependendo do número de valores faltantes e da importância da variável, pode-se optar por:

*   Descartar a variável (coluna) inteira
*   Descartar somente as linhas com valor faltante
*   Substituir por um valor específico (média, mediana, valor mais frequente etc.)

Por exemplo, para a coluna `Embarked`:

In [88]:
# Substituindo valores faltantes pela moda (valor mais frequente)
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

Algumas colunas podem não adicionar valor ao modelo. Por exemplo, a coluna `PassengerId` ou `Name` pode ser removida:

In [89]:
titanic_data.drop(['PassengerId', 'Name'], axis=1, inplace=True)

### 4.2 Convertendo Tipos de Dados

A coluna `Sex` pode ser codificada para 0 ou 1

In [90]:
# Convertendo a coluna 'Sex' para valores numéricos
titanic_data['Sex'] = titanic_data['Sex'].map({'male': 0, 'female': 1})

## 5. Seleção de Características

Como o foco do trabalho é o método de regularização, podemos selecionar as caracteristicas de forma mais simples, imaginando as caracteristicas que mais tem impacto na analise

In [91]:
selected_features = ['Pclass', 'Sex', 'Age', 'Fare']

## 6. Análise de dados

A análise exploratória de dados é uma etapa crítica antes de qualquer modelagem. Ela ajuda a entender a distribuição, a tendência, os padrões e as relações entre os dados. O método `describe` é uma ferramenta útil para essa fase, mas é apenas o começo.

In [92]:
titanic_data[selected_features].describe()

,Pclass,Sex,Age,Fare
count,891.000000,891.000000,891.000000,891.000000
mean,2.308642,0.352413,29.699118,32.204208
std,0.836071,0.477990,13.002015,49.693429
min,1.000000,0.000000,0.420000,0.000000
25%,2.000000,0.000000,22.000000,7.910400
50%,3.000000,0.000000,29.699118,14.454200
75%,3.000000,1.000000,35.000000,31.000000
max,3.000000,1.000000,80.000000,512.329200


In [93]:
titanic_data.corr(numeric_only=True)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
Survived,1.000000,-0.338481,0.543351,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.338481,1.000000,-0.131900,-0.331339,0.083081,0.018443,-0.549500
Sex,0.543351,-0.131900,1.000000,-0.084153,0.114631,0.245489,0.182333
Age,-0.069809,-0.331339,-0.084153,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.035322,0.083081,0.114631,-0.232625,1.000000,0.414838,0.159651
Parch,0.081629,0.018443,0.245489,-0.179191,0.414838,1.000000,0.216225
Fare,0.257307,-0.549500,0.182333,0.091566,0.159651,0.216225,1.000000


## 7. Modelagem

Primeiro vamos organizar tudo que vamos utilizar para os 2 métodos

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = titanic_data[selected_features]
y = titanic_data['Survived']

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Regularização L1 (Lasso)

In [95]:
# Criando e ajustando o modelo com regularização L1 (lasso)
model_l1 = LogisticRegression(penalty='l1', solver='saga', random_state=0)
model_l1.fit(X_train_scaled, y_train)

# Fazendo previsões com o conjunto de teste
y_pred_train_l1 = model_l1.predict(X_train_scaled)
y_pred_test_l1 = model_l1.predict(X_test_scaled)

# Calculando a acurácia do modelo com regularização L1
accuracy_train_l1 = accuracy_score(y_train, y_pred_train_l1)
accuracy_test_l1 = accuracy_score(y_test, y_pred_test_l1)

Regularização L2 (Ridge)

In [96]:
# Criando e ajustando o modelo com regularização L2 (ridge)
model_l2 = LogisticRegression(penalty='l2', solver='saga', random_state=0)
model_l2.fit(X_train_scaled, y_train)

# Fazendo previsões com o conjunto de teste
y_pred_train_l2 = model_l2.predict(X_train_scaled)
y_pred_test_l2 = model_l2.predict(X_test_scaled)

# Calculando a acurácia do modelo com regularização L2
accuracy_train_l2 = accuracy_score(y_train, y_pred_train_l2)
accuracy_test_l2 = accuracy_score(y_test, y_pred_test_l2)

## 8. Avaliação do modelo

In [97]:
print(f"Acurácia do treino com L1: {accuracy_train_l1}, Acurácia do teste com L1: {accuracy_test_l1}")
print(f"Acurácia do treino com L2: {accuracy_train_l2}, Acurácia do teste com L2: {accuracy_test_l2}")

Acurácia do treino com L1: 0.797752808988764, Acurácia do teste com L1: 0.7988826815642458
Acurácia do treino com L2: 0.7963483146067416, Acurácia do teste com L2: 0.8044692737430168


É notável que as acurácias nos conjuntos de treino e teste são muito próximas para ambos os modelos, o que é um bom indicativo de que o modelo está generalizando bem para dados não vistos. Isso sugere que o overfitting, um problema comum em modelos de aprendizado de máquina, **foi efetivamente controlado** pelas técnicas de regularização utilizadas.

Ao comparar os dois métodos de regularização, podemos ver que ambos apresentaram desempenhos semelhantes, tanto no conjunto de treino quanto no de teste. No entanto, o modelo com regularização L2 (Ridge) teve uma acurácia ligeiramente melhor no conjunto de teste (aproximadamente 80.45%) em comparação com o modelo L1 (Lasso), que teve uma acurácia de aproximadamente 79.89%.

Com base nos resultados, podemos concluir que a regularização desempenhou um papel efetivo na prevenção do overfitting. Ambos os métodos de regularização, L1 e L2, provaram ser eficazes, com o método L2 mostrando uma leve vantagem em termos de acurácia no conjunto de teste. Isso sugere que, para este conjunto de dados específico e o problema de classificação em questão, a regularização L2 pode ser a escolha mais apropriada.

### 8.1 Testando com meus dados:

In [100]:
# Criando um DataFrame com os meus dados
eu = pd.DataFrame({
    'Pclass': [1],
    'Sex': [0],
    'Age': [22],
    'Fare': [30]
})

eu_scaled = scaler.transform(eu)

# Usando o modelo para fazer a previsão
survival_prediction = model_l2.predict(eu_scaled)
survival_probability = model_l2.predict_proba(eu_scaled)

# Exibindo os resultados
print(f"Previsão de sobrevivência: {survival_prediction[0]} (0 para não sobreviveu, 1 para sobreviveu)")
print(f"Probabilidade estimada de sobrevivência: {survival_probability[0][1] * 100:.2f}%")

Previsão de sobrevivência: 1 (0 para não sobreviveu, 1 para sobreviveu)
Probabilidade estimada de sobrevivência: 54.13%


Quase que eu não saio vivo...

### 9. Explorando Diferentes Parâmetros para a Regularização

Além do tipo de penalidade (l1 para Lasso e l2 para Ridge), também é possível ajustar o parâmetro C, que é o inverso da força de regularização. Valores menores especificam uma regularização mais forte.

Para isso, vamos usar algo como o `GridSearchCV` do `scikit-learn` para testar diferentes combinações desses parâmetros:

In [99]:
from sklearn.model_selection import GridSearchCV

# Definindo os parâmetros que você deseja testar
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['saga']  # saga funciona tanto para l1 quanto para l2
}

# Criando o modelo
logistic_model = LogisticRegression()

# Instanciando o GridSearchCV
grid_search = GridSearchCV(logistic_model, param_grid, cv=5)

# Ajustando o modelo
grid_search.fit(X_train_scaled, y_train)

# Exibindo os melhores parâmetros
print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f"Melhor pontuação: {grid_search.best_score_}")

Melhores parâmetros: {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}
Melhor pontuação: 0.7921501034177091


### 10. Conclusão

Ao longo deste projeto, exploramos várias etapas críticas no desenvolvimento de um modelo de aprendizado de máquina, desde o pré-processamento de dados até a otimização do modelo. O objetivo principal foi entender e aplicar técnicas de regularização na construção de um modelo de regressão logística usando o conjunto de dados do `Titanic`.

Nosso modelo foi ajustado utilizando dois métodos de regularização diferentes, L1 (Lasso) e L2 (Ridge), para comparar a eficácia em prevenir o overfitting e melhorar a generalização do modelo para dados não vistos. Ambos os métodos se mostraram eficazes, com o L2 tendo uma leve vantagem em termos de acurácia no conjunto de teste.

No tópico "Explorando Diferentes Parâmetros para a Regularização", utilizamos `GridSearchCV` para otimizar os parâmetros do modelo. O resultado mostrou que a melhor configuração para este conjunto de dados específico foi com `C=0.1`, utilizando penalidade `l1` e o solver `saga`. A melhor pontuação obtida foi aproximadamente 79.22%, o que indica que nosso modelo é robusto e está alinhado com as acurácias obtidas anteriormente.

É crucial notar que a otimização dos parâmetros, especialmente o ajuste fino do parâmetro de regularização C, teve um impacto significativo na performance do nosso modelo. Isso ilustra a importância da regularização e da seleção de parâmetros na construção de modelos de aprendizado de máquina eficazes.

Em resumo, este projeto demonstrou como a aplicação cuidadosa de técnicas de regularização pode levar a modelos de classificação robustos. A habilidade de comparar e ajustar diferentes tipos de regularização e parâmetros permitiu não apenas melhorar a performance do modelo, mas também ganhar insights valiosos sobre as nuances da regularização em modelos de aprendizado de máquina.